# Construct further information
coring date, tidezone, water depth, and GPS (long_utm, lat_utm)

In [ ]:
import os
import numpy as np
import pandas as pd

path = '~\\GeopolarLabor\\#Projekte\\WASA'
os.chdir(path)

In [ ]:
new_info = pd.read_excel('XRF\\data_composite\\WASA_Umrechnung_Kerntiefen_20190313.xlsx', sheet_name = 'WASA_cores_20181127', 
                         usecols = [7, 11, 12, 13, 14 ,15])
len(new_info)

226

In [ ]:
# the na in dpeth_nhn means 0 water depth
new_info.Depth_NHN.fillna(0, inplace = True)
# remove the cores lacking information
new_info = new_info.dropna(axis = 0, how= 'any').reset_index(drop = True)

len(new_info)

126

In [ ]:
# change column names
cols = [_.lower() for _ in new_info.columns]
cols[2] = 'core_id'
new_info.columns = cols

In [ ]:
# uppercase the core_id
new_info['core_id'] = [ _.upper() for _ in new_info.core_id]

# asign core_id as the index
new_info.set_index('core_id', inplace = True)

In [ ]:
new_info['date'] = ['{}-{}-{}'.format(_[:4], _[4:6], _[6:]) for _ in new_info.date.astype(int).astype(str)]

In [ ]:
dates =[]
zones = []
longs = []
lats = []
depths = []

for row in range(len(info_df)):
  core_id = info_df.core_id[row]
  
  dates.append(new_info.date[core_id])
  zones.append(new_info.tidezone[core_id])
  longs.append(new_info.long_utm[core_id])
  lats.append(new_info.lat_utm[core_id])
  
  # plus the water depth
  depths.append(info_df.composite_depth_mm[row] - new_info.depth_nhn[core_id] * 1000)

info_df['coring_date'] = dates
info_df['tidezone'] = zones
info_df['long_utm'] = longs
info_df['lat_utm'] = lats
info_df['depth_to_water_surface_mm'] = depths

export new infos and the old infos with header

In [ ]:
for col in info_df.columns:
  info_df[col].to_csv('database\\info.cleaned.{}.csv'.format(col), index = False, header = col)

In [ ]:
new_info.head()

,date,tidezone,depth_nhn,long_utm,lat_utm
core_id,,,,,
N02,2017-08-16,subtidal,0.0,384547.933355,5.959440e+06
N03,2017-08-16,subtidal,0.0,388593.066041,5.960124e+06
N05,2018-04-17,subtidal,0.0,389006.938382,5.958470e+06
N06,2017-09-28,subtidal,0.0,384565.722230,5.955993e+06
N10,2018-04-17,subtidal,0.0,389460.021660,5.956522e+06


# Composite depth adjustment
Considering the offset caused by the cap (4~10 mm), the composite depth is subtracted by 4 mm overall. This act is believed to minimize the depth uncertainty in the range of +/- 2 mm.

In [1]:
import os
import pandas as pd
import glob
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.style.use('seaborn-colorblind')
%matplotlib inline

path = '~\\GeopolarLabor\\#Projekte\\WASA'
os.chdir(path)

In [6]:
X = pd.read_csv('database\\info.cleaned.core_id.csv', index_col = 0, squeeze = True)

In [8]:
len(set(X))

92

In [ ]:
info_cols = ['composite_id', 'core_id', 'core_section', 'composite_depth_mm', 'original_depth_mm']  # probably I can add other information such as tidezone

info_df = pd.DataFrame()
for col in info_cols:
  X = pd.read_csv('database\\info.cleaned.{}.csv'.format(col), squeeze = True)
  info_df[col] = X
  del X

In [ ]:
info_df.composite_depth_mm = info_df.composite_depth_mm - 4
info_df.composite_depth_mm.to_csv('database\\info.cleaned.composite_depth_mm.csv', index = False, header = True)

In [ ]:
info_df.head()

,composite_id,core_id,core_section,composite_depth_mm,original_depth_mm
0,N10_000005,N10,N10-5,1,5.0
1,N10_000007,N10,N10-5,3,7.0
2,N10_000009,N10,N10-5,5,9.0
3,N10_000011,N10,N10-5,7,11.0
4,N10_000013,N10,N10-5,9,13.0


These negative depth poitnts all happen in the top section. I rekon this is caused by the inconsistent position of air-sediment boundary between core description and the operator doing the scan. Therefore, I just delete those datapoints and update the whole database.

## update the result data

In [ ]:
X_result_df = pd.read_csv('database\XRF_results.cleaned.all.csv')
X_all_df = pd.concat((info_df, X_result_df), axis = 1)

In [ ]:
X_all_df = X_all_df[X_all_df.composite_depth_mm >= 0]
X_all_df.iloc[:, 5:].shape

(170437, 59)

In [ ]:
X_all_df.iloc[:, 5:].to_csv('database\XRF_results.cleaned.all.csv', index = False)
for col in X_all_df.iloc[:, 5:].columns:
  X_all_df[col].to_csv('database\XRF_results.cleaned.{}.csv'.format(col), header = True, index = False)

In [ ]:
del X_result_df

## update the composite id and infos

In [ ]:
idS = []

for row in X_all_df.index:
  idS.append('{}_{:05}'.format(X_all_df.core_id[row], X_all_df.composite_depth_mm[row])) 
  
pd.Series(idS, name = 'composite_id').to_csv('database\\info.cleaned.composite_id.csv', index = False, header = True)

In [ ]:
len(idS)

170437

In [ ]:
for col in X_all_df.iloc[:, 1:4]:
  X_all_df[col].to_csv('database\\info.cleaned.{}.csv'.format(col), index = False, header = True)

The above thread didn't update the origianl_depth_mm, so I use XRF_results.cleaned.position_mm.csv to update it. Meanwhile, there are some other info wasn't updated, I'll do it below.

In [ ]:
pd.read_csv('database/XRF_results.cleaned.position_mm.csv', names = ['original_depth_mm'], squeeze = True)[1:].to_csv('database\\info.cleaned.original_depth_mm.csv', index = False, header = True)

In [ ]:
olds = ['data_original_dir', 'composite_spe_dir', 'depth_to_water_surface_mm', 'lat_utm', 'long_utm', 'tidezone', 'coring_date' ]

In [ ]:
info_old = pd.DataFrame()
for col in olds:
  X = pd.read_csv('database\\info.cleaned.{}.csv'.format(col), squeeze = True)
  info_old[col] = X
  del X

In [ ]:
filename = pd.read_csv('database/XRF_results.cleaned.filename.csv', squeeze = True)

In [17]:
info_old[info_old.composite_spe_dir.isin(new_dir)]

,data_original_dir,composite_spe_dir,depth_to_water_surface_mm,lat_utm,long_utm,tidezone,coring_date


In [ ]:
old_dir = []
for row in range(len(info_old)):
  old_dir.append(info_old.composite_spe_dir[row].split('\\')[-1])

info_old['old_dir'] = old_dir

new_dir = []
for row in range(len(filename)):
  new_dir.append(
      '_'.join(
          filename[row].split('\\')[-1].split('_')[1:]
      )
  )
new_dir = pd.Series(new_dir)

In [ ]:
info_new = info_old[info_old.old_dir.isin(new_dir)]

In [40]:
info_new.shape

(170437, 8)

In [38]:
info_new.head()

,data_original_dir,composite_spe_dir,depth_to_water_surface_mm,lat_utm,long_utm,tidezone,coring_date,old_dir
0,\\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...,\\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...,5.0,5.956522e+06,389460.02166,subtidal,2018-04-17,N10_000005.spe
1,\\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...,\\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...,7.0,5.956522e+06,389460.02166,subtidal,2018-04-17,N10_000007.spe
2,\\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...,\\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...,9.0,5.956522e+06,389460.02166,subtidal,2018-04-17,N10_000009.spe
3,\\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...,\\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...,11.0,5.956522e+06,389460.02166,subtidal,2018-04-17,N10_000011.spe
4,\\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...,\\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...,13.0,5.956522e+06,389460.02166,subtidal,2018-04-17,N10_000013.spe


In [ ]:
for col in info_new.columns[:-1]:
  info_new[col].to_csv('database\\info.cleaned.{}.csv'.format(col), index = False, header = True)

# Update section depth
The logic follows the composite depth correction, so the depth is only corrected by the scanning depth correction(4 mm).

In [4]:
original = pd.read_csv('database/info.cleaned.original_depth_mm.csv', index_col = 0)
original

,original_depth_mm
composite_id,
N10_00001,5.00
N10_00003,7.00
N10_00005,9.00
N10_00007,11.00
N10_00009,13.00
N10_00011,15.00
N10_00013,17.00
N10_00015,19.00
N10_00017,21.00


In [11]:
section = original - 4
section.columns = ['section_depth_mm']
section.to_csv('database/info.cleaned.section_depth_mm.csv')

# Delete one data point having negative section depth

## read in datsets

In [2]:
info_df = pd.read_csv('database/info.cleaned.all.csv', index_col = 0)
info_df.shape

(170437, 12)

In [7]:
result_df = pd.read_csv('database/XRF_results.cleaned.all.csv')
result_df.index = info_df.index
result_df.shape

(170437, 59)

In [8]:
result_df.head()

,Al,Ar,As,Ba,Bi,Br,Ca,Ca_ka_Cr_ka,Ca_ka+_Ca__ka,Ca_ka+Ca_kb,...,W_la,W_lb1,W_lb2,Zn,Zr,cps,filename,position_mm,sample_surface,validity
composite_id,,,,,,,,,,,,,,,,,,,,,
N10_00001,42,798,59,130,63,8,6267,15,91,35,...,203,0,0,18,1270,23234,\\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...,5.0,5.89,1
N10_00003,55,775,0,149,31,0,6327,38,51,0,...,195,0,0,0,1549,26714,\\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...,7.0,5.96,1
N10_00005,63,613,0,125,25,8,7200,0,123,0,...,55,93,51,8,1369,26761,\\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...,9.0,5.88,1
N10_00007,46,729,20,102,46,11,5992,0,61,0,...,148,0,4,14,1199,25705,\\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...,11.0,5.91,1
N10_00009,52,728,84,127,28,0,10409,0,135,27,...,233,204,62,15,968,27081,\\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...,13.0,5.94,1


## exclude the rows based on id

In [18]:
exclude_id = ['N49_01409']

info_df = info_df[~info_df.index.isin(exclude_id)]
result_df = result_df[~result_df.index.isin(exclude_id)]

print(info_df.shape)
print(result_df.shape)

(170436, 12)
(170436, 59)


## output updated datasets

In [20]:
info_df.to_csv('database/info.cleaned.all.csv')
result_df.to_csv('database/XRF_results.cleaned.all.csv')

for col in info_df.columns:
    info_df[col].to_csv('database/info.cleaned.{}.csv'.format(col), header = col)
    
for col in result_df.columns:
    result_df[col].to_csv('database/XRF_results.cleaned.{}.csv'.format(col), header = col)

# Check XRF setting and update the spe dataset
Make sure all XRF are scanned in 30 kV and 45 mA setting. Also check if the tube is Cr although the data points aren't scanned by Cr should be excluded by previous filtering.

In [41]:
info_spe_dir = pd.read_csv('database\\info.cleaned.composite_spe_dir.csv', squeeze = True, index_col = 0)
info_spe_dir.head()

composite_id
N10_00001    \\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...
N10_00003    \\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...
N10_00005    \\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...
N10_00007    \\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...
N10_00009    \\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...
Name: composite_spe_dir, dtype: object

In [62]:
for doc in info_spe_dir[::50]:      # I won't check every spectrum because the same section should use same setting
    with open(doc, 'r') as f:
        lines = f.readlines()
        tube = lines[10].split()[2]
        vol = lines[11].split()[2]
        cur = lines[12].split()[2]
        if (tube != 'Cr') or (vol != '30') or (cur != '45'):
            print(doc.split('\\')[-1][:-11])

Nice!!! The latest cleaned XRF data are all produced by the same setting (30 kV & 45 mA) and the Cr tube.

## rebuild the spe using the latest composite_id
According to the last script that updated the spe in database, construct_database_20190502.py, the spe data was asopted from data_composite\\WASA_all_xrf_spe_20190405.csv' and the spe was filtered by the composite_spe_dir from info.cleaned.composite_spe_dir.csv.

In [41]:
info_spe_dir = pd.read_csv('..\\..\\database\\info.cleaned.composite_spe_dir.csv', squeeze = True, index_col = 0)
info_spe_dir.head()

composite_id
N10_00001    \\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...
N10_00003    \\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...
N10_00005    \\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...
N10_00007    \\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...
N10_00009    \\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...
Name: composite_spe_dir, dtype: object

In [35]:
len(info_spe_dir)

170436

Clearly the info_spe_dir has been updated to the latest version.

In [39]:
spe_df = pd.read_csv('WASA_all_xrf_spe_20190405.csv')
spe_df.head()

,1,2,3,4,5,6,7,8,9,10,...,1017,1018,1019,1020,1021,1022,1023,1024,spe_dir,core_ID
0,0,0,0,0,0,0,0,0,0,0,...,83,84,100,81,77,90,74,2,\\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...,N10
1,0,0,0,0,0,0,0,0,0,0,...,92,77,87,102,97,83,85,10,\\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...,N10
2,0,0,0,0,0,0,0,0,0,0,...,84,68,96,80,72,73,92,23,\\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...,N10
3,0,0,0,0,0,0,0,0,0,0,...,98,84,82,81,99,80,100,26,\\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...,N10
4,0,0,0,0,0,0,0,0,0,0,...,80,76,95,81,82,76,85,12,\\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...,N10


In [36]:
spe_df.shape

(173713, 1026)

In [42]:
spe_df[spe_df.spe_dir.isin(info_spe_dir)].head()

,1,2,3,4,5,6,7,8,9,10,...,1017,1018,1019,1020,1021,1022,1023,1024,spe_dir,core_ID
0,0,0,0,0,0,0,0,0,0,0,...,83,84,100,81,77,90,74,2,\\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...,N10
1,0,0,0,0,0,0,0,0,0,0,...,92,77,87,102,97,83,85,10,\\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...,N10
2,0,0,0,0,0,0,0,0,0,0,...,84,68,96,80,72,73,92,23,\\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...,N10
3,0,0,0,0,0,0,0,0,0,0,...,98,84,82,81,99,80,100,26,\\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...,N10
4,0,0,0,0,0,0,0,0,0,0,...,80,76,95,81,82,76,85,12,\\10.110.16.10\GeopolarLabor\#Projekte\WASA\XR...,N10


In [43]:
spe_up = spe_df[spe_df.spe_dir.isin(info_spe_dir)].iloc[:, :-2]
spe_up.index = info_spe_dir.index
spe_up.shape

(170436, 1024)

The spe dataset is asigned by the composite_id, which is more convenient for futehr arrangement.

## Export the spe

In [52]:
spe_up.to_csv('database\\XRF_spe.cleaned.all.csv')